In [33]:
import bs4
import psycopg2
import logging
import lxml
import sys
import requests
import json
import csv
import urllib3

In [34]:
# Load data from data.json
data_file = open("data.json")
data_dict = json.load(data_file)

fields = data_dict["event"]["data_field"]
table_name = data_dict["table_name"]
list_field_name = [f"{field['field']}" for field in fields]


In [35]:
# Create get_logger() function

def get_logger():
    logger = logging.getLogger('my_logger')
    logger.setLevel(logging.DEBUG)
    logger.propagate=False
    
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(lineno)d : %(message)s')
    
    file_handler = logging.FileHandler(data_dict["file_log"])
    file_handler.setFormatter(formatter)
    file_handler.setLevel(logging.DEBUG)

    if not logger.hasHandlers():
        logger.addHandler(file_handler)
        
    return logger
    

In [36]:
# Init csv writer
csv_file = open(data_dict["file_csv"], 'w')
csv_writer = csv.DictWriter(csv_file, fieldnames=list_field_name)

# Reset file log
with open(data_dict["file_log"], 'w'): pass

# Init file log to track history
logger = get_logger()


TypeError: __init__() missing 1 required positional argument: 'fieldnames'

In [ ]:
# Create connection to DBSM

def create_connection():
    f = open('password.txt', 'r') # password to my dbms
    pw = f.read()
    f.close()
    connection = psycopg2.connect(
        host = data_dict["dbms_host"],
        database = "postgres",
        user = "postgres",
        password = pw,
        port = "5432"
    )
    connection.set_session(autocommit=True)
    return connection

connection = create_connection()
logger.info(f"Connected to {connection}")

In [ ]:
# Create a table in DBMS

def create_table_dbms():
    logger.info(f"Start create table {table_name}")
    list_field_name_field_type = [f'{field["field"]} {field["type"]} ' for field in fields]
    str_field_name_field_type = ', '.join(list_field_name_field_type)
    create_table_str = f"""
        create table if not exists {table_name} (
            {str_field_name_field_type}
        )
    """
    cur = connection.cursor()
    cur.execute(create_table_str)

create_table_dbms()

In [ ]:
# Crawl data from youtube and generate

total_videos = []
cnt = 1
def crawl_data(cnt):
    # info = requests.get(data_dict["event"]["urls"])
    html_file = open("yt_html_sample_3.html")
    soup = bs4.BeautifulSoup(html_file, features = 'html5lib')
    videos = soup.find_all("ytd-rich-item-renderer", {"class" : "style-scope ytd-rich-grid-row", "items-per-row" : "3", "lockup" : "true"})
    print(len(videos))
    
    for video in videos:
        try:
            dismissible_video = video.find("div", {"id" : "dismissible"})
        except:
            continue
        
        video_id = cnt
        cnt += 1
        
        try:
            video_name = dismissible_video.find("a", {"id" : "video-title-link"})["title"]
            if "\'" in video_name:
                video_name = video_name.replace("\'", "")
        except:
            video_name = "Unknown"
        
        try:
            video_image_link = dismissible_video.find("img", {"style" : "background-color: transparent;"})["src"]
        except:
            video_image_link = "Unknown"
            
        try:
            video_link = dismissible_video.find("a", {"id" : "thumbnail"})["href"]
        except:
            video_link = "Unknown"
            
        try:
            author_name = dismissible_video.find("a", {"id" : "avatar-link"})["title"]
            if "\'" in author_name:
                author_name = author_name.replace("\'", " ")
        except:
            author_name = "Unknown"
            
        try:
            author_image_link = dismissible_video.find("img", {"id" : "img"})["src"]
        except:
            author_image_link = "Unknown"
        
        try:
            author_link = dismissible_video.find("a", {"id" : "avatar-link"})["href"]
        except:
            author_link = "Unknown"
        
        video_dict = {
            "video_id" : video_id,
            "video_name" : video_name,
            "video_image_link" : video_image_link,
            "video_link" : video_link,
            "author_name" : author_name,
            "author_image_link" : author_image_link,
            "author_link" : author_link
        }
        
        total_videos.append(video_dict)
    html_file.close()
    
crawl_data(cnt)
print(len(total_videos))

144
144


In [ ]:
# Insert data into DB and file csv

def insert_database():
    logger.info(f"Start insert data to table {table_name}")
    
    list_field_name = [f"{field['field']}" for field in fields]
    result_field_name = ', '.join(list_field_name)
    
    list_insert = [f"({video['video_id']}, '{video['video_name']}', '{video['video_image_link']}', '{video['video_link']}', '{video['author_name']}', '{video['author_image_link']}', '{video['author_link']}')" for video in total_videos]
    result_insert = ', '.join(list_insert)
    
    sql_insert = f"""
        INSERT INTO {table_name} ({result_field_name})
        VALUES {result_insert};
    """
    cur = connection.cursor()
    cur.execute(sql_insert)
    
def insert_csv():
    csv_writer.writeheader()
    csv_writer.writerows(total_videos)

insert_database()
insert_csv()
csv_file.close()